In [1]:
import joblib 
import numpy as np
import pandas as pd 
from pathlib import Path
import matplotlib.pyplot as plt
import boost_histogram as bh
import hist
from hist import Hist 

In [2]:
pythia_pkls = {}
for period in ["A", "D", "E"]:
    pythiaA_pkl_path = f'/global/cfs/projectdirs/atlas/hrzhao/HEP_Repo/QG_Calibration/tmp/pythia{period}_pred.pkl'
    pythiaA_pkl_path = Path(pythiaA_pkl_path)
    pythia_pd = joblib.load(pythiaA_pkl_path)
    pythia_pkls[f'pythia{period}'] = pythia_pd[(pythia_pd["jet_nTracks"] > 1) & (pythia_pd["target"] != '-')] 


In [3]:
reweighting_input = []
reweighting_vars = ['jet_nTracks', 'jet_trackBDT', 'GBDT_newScore'] 
reweighting_input_keys = reweighting_vars + ['is_forward', 'pt_idx','event_weight', 'target']

for pythia_key, pythia_pd in pythia_pkls.items():
    # drop jets with truth parton id == -1 
    reweighting_input.append(pythia_pkls[pythia_key][reweighting_input_keys])
    
reweighting_input = pd.concat(reweighting_input)

In [4]:
reweighting_input.head()

,jet_nTracks,jet_trackBDT,GBDT_newScore,is_forward,pt_idx,event_weight,target
0,2.0,-0.192873,-1.726218,1.0,1,17.609434,1
0,11.0,-0.442511,-3.864761,1.0,5,0.002348,0
1,14.0,-0.290822,-2.973750,0.0,5,0.002348,0
2,39.0,0.295834,0.704581,1.0,5,0.004385,1
3,37.0,0.159987,0.019631,0.0,4,0.004385,1


In [ ]:
joblib.dump(reweighting_input, 'reweighting_input.pkl')

In [ ]:
reweighting_input = joblib.load('reweighting_input.pkl')

In [ ]:
def safe_array_divide(numerator, denominator):
    with np.errstate(divide='ignore', invalid='ignore'):
        ratio = np.true_divide(numerator, denominator)
        ratio = np.nan_to_num(ratio, nan=0, posinf=0, neginf=0)
    return ratio

In [ ]:
def make_hist(values, bins, weights):
    # assuming bins numpy array with (start, stop, n_edges)
    _hist = Hist(hist.axis.Regular(bins=len(bins)-1, start=bins[0], stop=bins[-1], overflow=True, underflow=True), 
                                storage=hist.storage.Weight())
    _hist.fill(values, weight=weights)
    factor = np.sum(_hist.values())
    _normed_hist = _hist / (factor * _hist.axes[0].widths)

    return _hist, _normed_hist

In [ ]:
def calculate_reweight_factor(values_forward, values_central, weights_forward, weights_central, bins): 
    _, normed_hist_forward = make_hist(values_forward, bins=bins, weights=weights_forward)
    _, normed_hist_central = make_hist(values_central, bins=bins, weights=weights_central)

    return safe_array_divide(numerator=normed_hist_forward.values(), denominator=normed_hist_central.values())

In [ ]:
def calculate_reweight_factor_np(values_forward, values_central, weights_forward, weights_central, bins): 
    hist_forward, _ = np.histogram(values_forward, bins=bins, weights=weights_forward, density=False)
    hist_central, _ = np.histogram(values_central, bins=bins, weights=weights_central, density=False)

    return safe_array_divide(numerator=hist_forward, denominator=hist_central)

In [ ]:
# import boost_histogram as bh
# hist = bh.Histogram(bh.axis.Regular(bins=10, start=0, stop=1))
# hist.fill(0.9)
# hist.fill([0.9, 0.3, 0.4])
# np_array = hist.view()
# hist.variances()


In [ ]:
# Compare numpy histogram and boost histogram 
custom_bins = np.linspace(0, 60, 61)
hist_ntrk_np, bins_ntrk_np = np.histogram(reweighting_input['jet_nTracks'], bins=custom_bins, weights=reweighting_input['event_weight'])

## Another way to make histogram and sum the weights to bin error  
idx_nrtk_np = np.digitize(reweighting_input['jet_nTracks'], bins = custom_bins) - 1

idx_nrtk_np[idx_nrtk_np == (len(custom_bins) - 1 )] = (len(custom_bins) - 2) # merge the overflow bins to the second last one 
hist_content_np = np.bincount(idx_nrtk_np, weights = reweighting_input['event_weight'])
hist_err_np = np.bincount(idx_nrtk_np, weights = reweighting_input['event_weight'] ** 2)

In [ ]:
_hist = Hist(hist.axis.Regular(bins=60, start=0, stop=60, overflow=True, underflow=True), 
                                storage=bh.storage.Weight())
_hist.fill(reweighting_input['jet_nTracks'], weight=reweighting_input['event_weight'])


In [ ]:
_hist.values()

In [ ]:
np.isclose(np.sqrt(normed_hist.variances()),  np.sqrt(_hist.variances()) / (factor * _hist.axes[0].widths))

In [ ]:
import boost_histogram as bh
hist_ntrk_bh = bh.Histogram(bh.axis.Regular(bins=60, start=0, stop=60, overflow=True, underflow=True), 
                            storage=bh.storage.Weight())
hist_ntrk_bh.fill(reweighting_input['jet_nTracks'], weight=reweighting_input['event_weight'])

# test 
# np.isclose(np.sum(reweighting_input['event_weight']), np.sum(hist_ntrk_bh.values(flow=True)))
# True 

factor = np.sum(hist_ntrk_bh.values())
normed = hist_ntrk_bh / (factor * hist_ntrk_bh.axes[0].widths)

In [ ]:
_reweighting_input = reweighting_input[reweighting_input['jet_nTracks'] <= 60]
hist_ntrk_np, bins_ntrk_np = np.histogram(_reweighting_input['jet_nTracks'], bins=custom_bins, weights=_reweighting_input['event_weight'], density=True)
plt.errorbar(x = normed.axes[0].centers, y = normed.values(), yerr=np.sqrt(normed.variances()), ls='', marker= '.')

In [ ]:
def get_reweight_factor_pd(reweighting_input, clf_type="new_GBDTscore"):
    reweight_factor = {}
    reweighting_vars = ['jet_nTracks', 'jet_trackBDT', 'GBDT_newScore'] 
    label_pt_bin = np.array([500, 600, 800, 1000, 1200, 1500, 2000])
    HistBins = {
        'jet_nTracks' : np.linspace(0, 60, 61),
        'jet_trackBDT' : np.linspace(-1.0, 1.0, 101),
        'GBDT_newScore' : np.linspace(-1.0, 1.0, 101),
    }


    for pt_idx, pt in enumerate(label_pt_bin[:-1]):
        print(pt)
        reweight_factor[pt] = {}
        reweighting_input_at_pt = reweighting_input[reweighting_input['pt_idx'] == pt_idx]

        forward_quark_at_pt = reweighting_input_at_pt.loc[(reweighting_input_at_pt['target'] == 0) & 
                                                        (reweighting_input_at_pt['is_forward'] == 1)]
        central_quark_at_pt = reweighting_input_at_pt.loc[(reweighting_input_at_pt['target'] == 0) & 
                                                        (reweighting_input_at_pt['is_forward'] == 0)]
        forward_gluon_at_pt = reweighting_input_at_pt.loc[(reweighting_input_at_pt['target'] == 1) & 
                                                        (reweighting_input_at_pt['is_forward'] == 1)]
        central_gluon_at_pt = reweighting_input_at_pt.loc[(reweighting_input_at_pt['target'] == 1) & 
                                                        (reweighting_input_at_pt['is_forward'] == 0)]   

        for var in reweighting_vars:
            bin_var = HistBins[var]

            quark_factor = calculate_reweight_factor(values_forward=forward_quark_at_pt[var], values_central=central_quark_at_pt[var],
                                                     weights_forward=forward_quark_at_pt['event_weight'], weights_central=central_quark_at_pt['event_weight'],
                                                     bins=bin_var)

            gluon_factor = calculate_reweight_factor(values_forward=forward_gluon_at_pt[var], values_central=central_gluon_at_pt[var],
                                                     weights_forward=forward_gluon_at_pt['event_weight'], weights_central=central_gluon_at_pt['event_weight'],
                                                     bins=bin_var)
            
            reweight_factor[pt][var]={
                'quark_factor' : quark_factor, 
                'gluon_factor' : gluon_factor, 
            }

    return reweight_factor

In [ ]:
test_reweight_factor = get_reweight_factor_pd(reweighting_input)
# 4 mins 

In [ ]:
test_reweight_factor3 = get_reweight_factor(reweighting_input)
# 4 mins 

In [ ]:
joblib.dump(test_reweight_factor, 'test_reweight_factor2.pkl')

In [ ]:
test_reweight_factor2 = test_reweight_factor

In [ ]:
np.allclose(test_reweight_factor3[500]['jet_trackBDT']['quark_factor'], test_reweight_factor2[500]['jet_trackBDT']['quark_factor'])

In [ ]:
pythia_pkls['pythiaE']

In [ ]:
def attach_reweight_factor(sample, clf_type):
    assert clf_type in ['new_MLPprob', 'new_GBDTscore']

    if clf_type == 'new_MLPprob':
        clf_range = (0, 1) 
    if clf_type == 'new_GBDTscore':
        clf_range = (-5.0, 5.0) 

    features = [*sample.columns[:6]] + [clf_type] 
    HistBins = {
        features[0] : np.linspace(0, 2000, 61), 
        features[1] : np.linspace(-2.5, 2.5, 51),
        features[2] : np.linspace(0, 60, 61),
        features[3] : np.linspace(0, 0.4, 61), 
        features[4] : np.linspace(0, 0.4, 61), 
        features[5] : np.linspace(-1.0, 1.0, 51),
        clf_type : np.linspace(clf_range[0], clf_range[1], 51)
    }
    label_vars = ['jet_nTracks', 'jet_trackBDT', clf_type]

    # Initialize all the vars 
    for var in label_vars:
        sample[f'{var}_quark_reweighting_weights'] = sample['event_weight'].copy()
        sample[f'{var}_gluon_reweighting_weights'] = sample['event_weight'].copy()

    reweighted_sample = []
    for pt_idx, pt in enumerate(tqdm(label_pt_bin[:-1])):
        sample_pt = sample[sample['pt_idx'] == pt_idx]  # Get the pt slice 
        _sample = sample_pt
        
        forward_quark = _sample[(_sample['is_forward']==1) &(_sample['target']==0)]
        forward_gluon = _sample[(_sample['is_forward']==1) &(_sample['target']==1)]
        central_quark = _sample[(_sample['is_forward']==0) &(_sample['target']==0)]
        central_gluon = _sample[(_sample['is_forward']==0) &(_sample['target']==1)]

        for var in label_vars:
            bin_var = HistBins[var]
            hist_forward_quark, _ = np.histogram(forward_quark[var], bins=bin_var, weights=forward_quark['event_weight'])
            hist_central_quark, _ = np.histogram(central_quark[var], bins=bin_var, weights=central_quark['event_weight'])
            hist_forward_gluon, _ = np.histogram(forward_gluon[var], bins=bin_var, weights=forward_gluon['event_weight'])
            hist_central_gluon, _ = np.histogram(central_gluon[var], bins=bin_var, weights=central_gluon['event_weight'])

            quark_factor = safe_array_divide(numerator=hist_forward_quark, denominator=hist_central_quark)
            gluon_factor = safe_array_divide(numerator=hist_forward_gluon, denominator=hist_central_gluon)

            new_var_idx = pd.cut(_sample[var], bins=bin_var, right=False, labels=False)  # Binned feature distribution 
            for i, score in enumerate(bin_var[:-1]): # Loop over the bins 
                mod_idx = np.where(new_var_idx == i)[0]
                _sample.iloc[mod_idx, _sample.columns.get_loc(f'{var}_quark_reweighting_weights')] *= quark_factor[i]
                _sample.iloc[mod_idx, _sample.columns.get_loc(f'{var}_gluon_reweighting_weights')] *= gluon_factor[i]
            
        reweighted_sample.append(_sample)

    return pd.concat(reweighted_sample)


